In [47]:
import pandas as pd
import openreview
import json

from collections import OrderedDict
from typing import Any, Callable, Iterable, List, Dict, Optional

In [48]:
def save_json(data,fileName,convert = True):
    with open(f"{fileName}.json","w") as file:
        data_converted = data.to_json() if convert else data
        json.dump(data_converted,file,indent=4)

In [38]:
import openreview.api as openreview_api

v2_cl  = openreview_api.OpenReviewClient("https://api2.openreview.net")
invitations = v2_cl.get_all_invitations(prefix='ICLR.cc/2018/Conference')

print([inv.id for inv in invitations])

submit_2 = v2_cl.get_all_notes(
    invitation='ICLR.cc/2018/Conference/-/Blind_Submission',
    details='directReplies'
) 

OpenReviewException: {'name': 'ValidationError', 'message': 'request requires at least one of id, ids, invitation, replyForum, replyInvitation, invitee, edit.note.forum, edit.invitation.id, edit.group.id, message.groups.param.inGroup, tag.forum, tag.note as parameter (2025-10-25-6278303)', 'status': 400, 'details': {'path': 'request', 'reqId': '2025-10-25-6278303'}}

In [16]:
print(submit_2)

[]


In [61]:
def fetch_submissions_iclr2018():
    # Try API v2 first (will likely return 0 for 2018)
    try:
        import openreview.api as openreview_api
        v2 = openreview_api.OpenReviewClient(baseurl="https://api2.openreview.net")
        notes_v2 = v2.get_all_notes(invitation="ICLR.cc/2022/Conference/-/Blind_Submission")
        if notes_v2:
            print("bitch1")
            return notes_v2
    except Exception:
        print("bitch2")
        # pass

    # Fallback to API v1 for legacy venues
    import openreview
    v1 = openreview.Client(baseurl="https://api.openreview.net")
    print("bitch 3")
    return v1.get_all_notes(invitation="ICLR.cc/2022/Conference/-/Blind_Submission")

notes = fetch_submissions_iclr2018()
print(notes[0])
with open("2022.json","w") as f:
    json.dump(notes[0].to_json(),f,indent=4)
print("ICLR 2018 notes:", len(notes))


bitch 3


Getting V1 Notes: 100%|█████████▉| 2614/2617 [00:00<00:00, 4630.73it/s]

{'cdate': 1632875730339,
 'content': {'_bibtex': '@inproceedings{\n'
                        'rajkumar2022a,\n'
                        'title={A Theory of Tournament Representations},\n'
                        'author={Arun Rajkumar and Vishnu Veerathu and Abdul '
                        'Bakey Mir},\n'
                        'booktitle={International Conference on Learning '
                        'Representations},\n'
                        'year={2022},\n'
                        'url={https://openreview.net/forum?id=zzk231Ms1Ih}\n'
                        '}',
             'abstract': 'Real-world tournaments are almost always '
                         'intransitive. Recent works have noted that '
                         'parametric models which assume  $d$ dimensional node '
                         'representations can effectively model intransitive '
                         'tournaments. However, nothing is known about the '
                         'structure of the clas

In [ ]:
# api2_venueid = 'NeurIPS.cc/2024/Conference' #In 2024, NeurIPS used API 2

client = openreview.Client(baseurl='https://api.openreview.net')  # API v1

# submissions = client.get_all_notes(
#     invitation='ICLR.cc/2018/Conference/-/Blind_Submission',
#     details='directReplies',   # embed replies (reviews, comments, etc.)
# )
api2_venueid = 'ICLR.cc/2018/Conference' #In 2024, NeurIPS used API 2

api2_venue_group = v2_cl.get_group(api2_venueid)
api2_venue_domain = api2_venue_group.domain
print(api2_venue_domain) 
#Output: NeurIPS.cc/2024/Conference

api1_venueid = 'ICLR.cc/2018/Conference' #In 2022, NeruIPS used API1
api1_venue_group = client.get_group(api1_venueid)
api1_venue_domain = api1_venue_group.domain
print(api1_venue_domain) 
#Output: None

NameError: name 'v2_cl' is not defined

In [37]:
v1_cl = openreview.Client(baseurl="https://api.openreview.net")

submit_1 = v1_cl.get_all_notes(
    invitation='ICLR.cc/2018/Conference/-/Blind_Submission',
    details='directReplies'
) 

In [30]:
# print(submit_1[])

with open(f"paper_v1.json","w") as file:
  json.dump(submit_1[30].to_json(),file,indent=4)

In [ ]:
url = "ICLR.cc/2022/Conference/-/Blind_Submission"

client = openreview.Client(baseurl='https://api.openreview.net')  # API v1
submissions = client.get_all_notes(invitation=url,details="directReplies")

paper0 = submissions[0]
print(type(paper0))

Getting V1 Notes: 100%|█████████▉| 2614/2617 [00:01<00:00, 2515.87it/s]

<class 'dict'>


In [ ]:
save_json(paper0,"det",False)

<h1> hello

In [11]:
# Get the forum of the paper 
reviews = client.get_notes(forum=paper0.forum)
review1 = reviews[1]

In [18]:
for i in range(len(reviews)):
    with open(f"review_{i}.json","w") as file:
        json.dump(reviews[i].to_json(),file,indent=4)

In [76]:
def threadify(
    reviews: Iterable[Any],
    *,
    to_dict: Callable[[Any], Dict[str, Any]] = lambda x: x.to_json() if hasattr(x, "to_json") else dict(x),
    id_key: str = "id",
    parent_key: str = "replyto",
    root_id: Optional[Any] = None,            # if None, we’ll infer from the first item’s .forum if present
    children_key: str = "Sub(s)",             # keep your existing key name
    keep_order: bool = True                   # preserve input order within siblings
) -> List[Dict[str, Any]]:
    """
    Build a nested comment tree of arbitrary depth.

    - reviews: list/iterable of objects or dicts
    - to_dict: how to turn each item into a dict
    - id_key: field with the unique id
    - parent_key: field pointing to parent id
    - root_id: id considered the top-level parent (items replying to this become roots).
               If None, and the first review has attribute `.forum`, that value is used.
    - children_key: key used to store children
    - keep_order: if True, siblings appear in the same order as input
    """
    reviews = list(reviews)
    if not reviews:
        return []

    # Infer root_id from the first element’s `.forum` if not provided
    if root_id is None and hasattr(reviews[0], "forum"):
        root_id = getattr(reviews[0], "forum")

    # Normalize to dicts while preserving input order index (for stable sibling ordering)
    norm: List[Dict[str, Any]] = []
    for idx, item in enumerate(reviews):
        d = to_dict(item)
        d.setdefault(children_key, [])
        d["_order_index"] = idx  # hidden key for ordering; stripped later
        norm.append(d)

    # Index by id
    by_id: "OrderedDict[Any, Dict[str, Any]]" = OrderedDict()
    for d in norm:
        key = d.get(id_key)
        if key is not None:
            by_id[key] = d

    # Attach children to parents where possible
    roots: List[Dict[str, Any]] = []
    for d in norm:
        pid = d.get(parent_key)
        # Root if it replies to the root_id OR if parent is missing/unknown
        if pid == root_id or pid not in by_id:
            roots.append(d)
        else:
            parent = by_id[pid]
            parent.setdefault(children_key, []).append(d)

    # Optionally sort siblings by original input order
    def sort_children(node: Dict[str, Any]):
        if children_key in node and node[children_key]:
            node[children_key].sort(key=lambda x: x["_order_index"])
            for c in node[children_key]:
                sort_children(c)

    if keep_order:
        for r in roots:
            sort_children(r)
        roots.sort(key=lambda x: x["_order_index"])

    # Strip helper keys
    def cleanup(node: Dict[str, Any]):
        node.pop("_order_index", None)
        for c in node.get(children_key, []):
            cleanup(c)

    for r in roots:
        cleanup(r)

    return roots

def getSinglePaperDetail(paperForumID):
    reviews = client.get_notes(forum=paperForumID)
    tree = threadify(
        reviews,
        id_key="id",
        parent_key="replyto",
        children_key="Sub(s)",   # keeps your existing name
        # root_id=reviews[0].forum   # optional; inferred if reviews[0] has .forum
    )
    # Just a bit of clean up
    del tree[0]["Sub(s)"]

    # Actual URL to the paper
    # tree[0].content["pdf"] = f"https://openreview.net{tree[0].content["pdf"]}"
    # Starting point
    paper_reviews = tree[0]

    # Get the decision
    for revObj in reviews[1:]:
        rev = revObj.to_json()
        if rev["content"].get("decision"):
            paper_reviews["decision"] = rev["content"].get("decision")
            break
    
    # Add all comments and reviews
    paper_reviews["reviews"] = tree[1:]

    return paper_reviews
    

935

In [ ]:
counter = 0
total = len(submissions)
with open("iclr2018.jsonl","w",encoding="utf-8") as f:
    for paper in submissions:
        print(f"({int(100*(counter/total))}%) Processing Paper: {counter} out of {total}")
        f.write(json.dumps(getSinglePaperDetail(paper.forum)) + "\n")
        counter +=1
        

running paper: 0 out of 935 (0%)
Retrying request: GET /notes?forum=ryzm6BATZ, response: <urllib3.response.HTTPResponse object at 0x12601a6b0>, error: None
running paper: 1 out of 935 (0%)
running paper: 2 out of 935 (0%)
running paper: 3 out of 935 (0%)
running paper: 4 out of 935 (0%)
running paper: 5 out of 935 (0%)
running paper: 6 out of 935 (0%)
running paper: 7 out of 935 (0%)
running paper: 8 out of 935 (0%)
running paper: 9 out of 935 (0%)
running paper: 10 out of 935 (1%)
running paper: 11 out of 935 (1%)
running paper: 12 out of 935 (1%)
running paper: 13 out of 935 (1%)
running paper: 14 out of 935 (1%)
running paper: 15 out of 935 (1%)
running paper: 16 out of 935 (1%)
running paper: 17 out of 935 (1%)
running paper: 18 out of 935 (1%)
running paper: 19 out of 935 (2%)
running paper: 20 out of 935 (2%)
running paper: 21 out of 935 (2%)
running paper: 22 out of 935 (2%)
running paper: 23 out of 935 (2%)
running paper: 24 out of 935 (2%)
running paper: 25 out of 935 (2%)
run

KeyboardInterrupt: 

In [ ]:
data = []

for note in submissions:
    n = note.to_json()        # convert Note → dict
    c = n.get('content', {})  # the main content fields
    print(n)
    break
    # Extract core info
    data.append({
        'title': c.get('title', ''),
        'authors': ', '.join(c.get('authors', [])),
        'keywords': ', '.join(c.get('keywords', [])),
        'abstract': c.get('abstract', ''),
        'pdf_url': f"https://openreview.net{c.get('pdf', '')}",
        'forum_url': f"https://openreview.net/forum?id={n.get('forum', '')}",
        'invitation': n.get('invitation', ''),
        'signatures': ', '.join(n.get('signatures', [])),
        'created_date': pd.to_datetime(n.get('cdate', 0), unit='ms'),
        'last_modified': pd.to_datetime(n.get('tmdate', 0), unit='ms')
    })

# 4️⃣ Create a DataFrame
# df = pd.DataFrame(data)

# # 5️⃣ Display (cleaner formatting)
# pd.set_option('display.max_colwidth', 120)
# display(df.head())
# print(f"\nTotal submissions: {len(df)}")

{'id': 'ryzm6BATZ', 'original': 'B1zmTBAT-', 'cdate': 1518730189355, 'pdate': None, 'odate': None, 'mdate': None, 'tcdate': 1508953370259, 'tmdate': 1644493022819, 'ddate': None, 'number': 93, 'content': {'title': 'Image Quality Assessment Techniques Improve Training and Evaluation of Energy-Based Generative Adversarial Networks', 'abstract': "We propose a new, multi-component energy function for energy-based Generative Adversarial Networks (GANs) based on methods from the image quality assessment literature. Our approach expands on the Boundary Equilibrium Generative Adversarial Network (BEGAN) by outlining some of the short-comings of the original energy and loss functions. We address these short-comings by incorporating an l1 score, the Gradient Magnitude Similarity score, and a chrominance score into the new energy function. We then provide a set of systematic experiments that explore its hyper-parameters. We show that each of the energy function's components is able to represent a